In [ ]:
train_data = [
    ("Modern environmental culture first manifested itself in Montreal in the work of the Quebecois Art Deco architect-engineer Ernest Cormier, primarily through his Université de Montreal, constructed between 1928 and 1955.", "Université de Montreal", "ARCHITECTURE"),
    ("the Place Ville Marie (1958), an office and shopping complex by I.M. Pei", "Place Ville Marie", "ARCHITECTURE"),
    ("the merchandise mart and rooftop hotel, known as the Place Bonaventure, built to the designs of Ray Affleck in 'corduroy' bush-hammered concrete in 1964.", "Place Bonaventure", "ARCHITECTURE"),
    ("This exhibition was dominated by two monumental works, the first being Buckminster Fuller's US Pavilion, which took the unique form of an enormous geodesic sphere.", "US Pavilion", "ARCHITECTURE"),
    ("The other prominent work, which was also predicated on tetrahedral geometry, was Moshe Safdie's multistorey experimental housing complex, known as Habitat '67", "Habitat '67", "ARCHITECTURE"),
    ("Toronto's entry into modernity began in 1958 with the international competition for Toronto City Hall", "Toronto City Hall", "ARCHITECTURE"),
    ("However, the first move towards a regionally inflected Canadian architecture appeared in Toronto in the work of Ron Thom, whose part-Brutalist, part-late Gothic Revival enclave of Massey College was constructed in the built-up area of the city in 1963", "enclave of Massey College", "ARCHITECTURE"),
    ("This was followed in 1965 by the equally Brutalist, organically planned Scarborough College", "Scarborough College", "ARCHITECTURE")
]

test_data = [
    ("On the west coast, the first figure of stature to emerge during the post-war period was Arthur Erickson, whose Simon Fraser University (1962-72), conceived as a landscaped megastructure, was integrated into the rising site of a small mountain near Vancouver.", "Simon Fraser University", "ARCHITECTURE"),
    ("Erickson followed this achievement with two other megastructural works: his bridge-like design for the University of Lethbridge in Alberta (1972), and his Robson Square complex, completed as an axial spine running through the centre of Vancouver in 1986.", "University of Lethbridge", "ARCHITECTURE"),
    ("Erickson followed this achievement with two other megastructural works: his bridge-like design for the University of Lethbridge in Alberta (1972), and his Robson Square complex, completed as an axial spine running through the centre of Vancouver in 1986.", "Robson Square complex", "ARCHITECTURE"),
    ("Even so, the Ecole des Beaux-Arts still exercised an influence, mainly through the activity of French architects who, at the turn of the century, were appointed as professors in a number of leading American universities", "Ecole des Beaux-Arts", "ORG")
]


In [ ]:
import pandas as pd

def prepare_ner_training_data(data):
    training_data = []

    for sentence, entity_text, entity_label in data:
        sentence = sentence.lower()
        entity_text = entity_text.lower()

        sentence = sentence.replace(",", "")
        sentence = sentence.replace(".", "")
        sentence = sentence.replace(";", "")
        sentence = sentence.replace("!", "")
        sentence = sentence.replace("?", "")

        tokens = sentence.split()
        labels = ["O"] * len(tokens)

        # Find the start and end index of the entity text in the tokens
        entity_tokens = entity_text.split()
        for i in range(len(tokens) - len(entity_tokens) + 1):
            if tokens[i:i + len(entity_tokens)] == entity_tokens:
                labels[i] = f"B-{entity_label}"
                for j in range(1, len(entity_tokens)):
                    labels[i + j] = f"I-{entity_label}"

        training_data.append({"tokens": tokens, "ner_tags": labels})

    return training_data

# Example usage
training_data = prepare_ner_training_data(train_data)
test_data = prepare_ner_training_data(test_data)

In [ ]:
import pandas as pd

train_df = pd.DataFrame(training_data)
test_df = pd.DataFrame(test_data)

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

label_list = ['O','B-ARCHITECTURE','I-ARCHITECTURE','B-ORG','I-ORG']
label_encoding_dict = {'O': 0, 'B-ARCHITECTURE': 1, 'I-ARCHITECTURE': 2, 'B-ORG': 3, 'I-ORG': 4}

task = "ner"
model_checkpoint = "dslim/distilbert-NER"
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=264621db728a42db03b1cdffad8a7feaf2ecaf26f4df75d59247ff40b0af898e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
import numpy as np


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True)

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('arch-ner.model')

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dslim/distilbert-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.583160,0.000000,0.000000,0.000000,0.889908
2,No log,0.508250,0.000000,0.000000,0.000000,0.889908
3,No log,0.519958,0.000000,0.000000,0.000000,0.889908


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are i

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained('./arch-ner.model/')

input_text = '''
Two works by Patkau Architects dating from 2016 testify
to their continual inventiveness. The first of these is the
Audain Art Museum, projected as a bridge-like, pitched-roof
gallery spanning over the flood plan of Fitzsimmons Creek in
Whistler, British Columbia. The second is a visitors’ centre
close to the 200-year-old Fort York National Historic Site,
located adjacent to an elevated expressway feeding the
centre of Toronto. Parallel to the monumental autoroute, this
visitors’ centre is clad with inclined plates of corten steel
that house a reception area, a café, an orientation space
and a semi-subterranean ‘time-tunnel’ containing a video
history of the fort.
One of the most significant Ontario practices of the 21st
century is that of Brigitte Shim and Howard Sutcliffe, who
opened their office in Toronto in 1994. Like Patkau Architects
they are acutely aware that their work is located in a vast
northern territory subject to the stresses of a harsh climate.
Shim has characterized the context of their practice in the
following terms:
In Canada, we occupy an enormous landmass with a tiny
population. Whether one lives in the countryside or in the
city, the mythological Canadian landscape permeates our
lives. … The majority of our work is located at the bottom
edge of the Canadian Shield, described as a stone necklace
of ancient metamorphic rock wrapping around Hudson’s
Bay.
2
Their first work on the Canadian Shield was their
Moorelands Camp dining hall of 2002, which took the form
of a large timber shed, built on Lake Kawagama at
Haliburton. It was designed for a long-established summer
camp catering to the needs of urban youth, and for this
reason it was fitted with large, top-hung louvred doors,
enabling the hall to be closed down completely at the end of
the season. Shim-Sutcliffe would parallel this exercise in
modular timber construction in their Harrison Island Camp
[371], built as a vacation house for their own occupation, on
Georgian Bay, Ontario, in 2010. This house was largely
composed of structural insulated panels, which were
shipped to the rocky site by barge and then assembled into
position.
'''
paragraph = ' '.join([line for line in input_text.split('\n') if line != ''])



tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

model = AutoModelForTokenClassification.from_pretrained('./arch-ner.model/', num_labels=len(label_list))
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

words = tokenizer.batch_decode(tokens['input_ids'])
pd.DataFrame({'ner': predictions, 'words': words})

,ner,words
0,O,[CLS]
1,O,Two
2,O,works
3,O,by
4,O,Pat
...,...,...
457,O,assembled
458,O,into
459,O,position
460,O,.
